# Classification of Parkinsson patients from EEG signals #
Ekaterina Lavrova 

## Import Packages ##

In [ ]:
#import packages 
import os
import numpy as np
import pandas as pd 
import mne 
from mne.preprocessing import ICA
from scipy.signal import welch 
from sklearn.metrics import accuracy_score,recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler  

 ## Import Data ##

In [ ]:
#import data
file_path = "C:/Users/katja/Programing_project/Data" 
raw_list = []

for file in os.listdir(file_path):
    raw = mne.io.read_raw_bdf(os.path.join(file_path,file), preload = True,verbose = False) 
    raw_list.append(raw)

## Filtering ##

In [ ]:
#Filtering 
raw_test = raw_list[1]
raw_test.plot(start = 0, duration = 30,scalings = dict(eeg=200e-6)) # Sclaed plot of file with index 1 

for raw in raw_list: 
    raw.filter(1.0,40.0, verbose = False) # Bandpass filtering 
    raw.notch_filter(50.0, verbose = False) # Notch filtering ( 50 Hz european standards, change to 60 Hz in USA)
    raw.set_eeg_reference('average', verbose = False) #Rereferencing using average method

raw_test.plot(start = 0, duration = 30,scalings = dict(eeg=200e-6)) #Plot after filtering

## Prepare ICA and data for artifact removal ##

In [ ]:
ica_list = [] #Empty list for ICA components 

for raw in raw_list:
    montage = mne.channels.make_standard_montage('standard_1020') # Give channels positions according to the 10 20 system 

    # Mark EXG channels as NA 
    raw.set_channel_types({
        'EXG1':'misc', 'EXG2':'misc', 'EXG3':'misc', 'EXG4':'misc',
        'EXG5':'misc', 'EXG6':'misc', 'EXG7':'misc', 'EXG8':'misc'
    })
    raw.set_montage(montage)

# Configure and fit ICA to the data
for raw in raw_list:
    ica = ICA(n_components=15, method='fastica', random_state=42, verbose = False)
    ica.fit(raw, verbose = False )
    ica_list.append(ica)

## Plot ICA components to identify artifacts ##

In [ ]:
# Creates variable  that only include one file
raw_test = raw_list[1]
ica_test = ica_list[1]

# Plot components of one file 
ica_test.plot_components()

# Plot overlay of raw signal with specific ICA components excluded for one file 
ica_test.plot_overlay(raw_test, exclude=[0])  # exclude component 0 in file idx 1
ica_test.plot_overlay(raw_test, exclude=[1])  # exclude component 1 in file idx 1

## Apply ICA filtering to remove artifacts ## 

In [ ]:
#Apply ICA filtering 
processed_raw = []

#Combine ICA and and the raw datafiles 
for ica, raw in zip(ica_list, raw_list):
    ica.exclude = [0,1] #exclude components 

    raw_clean = ica.apply(raw.copy(), verbose = False) #Create copies of each raw file xith ica applied 
    processed_raw.append(raw_clean) # add raw_copies to list

# Plot one fiel from the processed list to compare with unfiltered plot
raw = processed_raw [1]
raw.plot (start = 0, duration = 30, scalings=dict(eeg=200e-6))

## Defining Feature Functions ##

In [ ]:

#Computing shannon entropy 
def shannon_entropy(signal):
    pdf, _ = np.histogram(signal, bins=50, density=True)
    pdf = pdf[pdf > 0]
    return -np.sum(pdf * np.log2(pdf))

#Computing bandpower 
def compute_bandpowers(signal, sfreq):
    freqs, psd = welch(signal, sfreq, nperseg=len(signal))

#Defining band frequencies 
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 12),
        'beta':  (12, 30),
        'gamma': (30, 45)
    }

    bp = {}
    for band, (low, high) in bands.items(): #Looping through bands to define frequency range 
        idx = (freqs >= low) & (freqs <= high) #boolean array indicating if the frequencies in the segment fall inside a specific frequency band 
        bp[band] = np.trapz(psd[idx], freqs[idx]) # Calculates psd for the frequencies in the specific band and segment 
    return bp

## Apply feature extraction ##

In [ ]:
#Initiating empty list for features 
subject_features = []

file_path = "c:\\Users\\katja\\Programing_project\\Data" #Indicate file path to datafiles 

files = [file for file in os.listdir(file_path) ] #Iterate over files in file path and save filename in "files"

# Pair each processed file with its' filename and assign a group and subject based on the file name 
for raw_clean, file in zip(processed_raw, files):
    group = "healthy" if "sub-hc" in file else 'pd' # if filename contains "sub-hc" assign to healthy
    subject = file.split('_')[0].replace('sub-','') # Keep everything before _ in file name and remove the "sub" letters to get subject
    
    #Epoch the data 
    epochs =mne.make_fixed_length_epochs(
        raw_clean, 
        duration = 20, # 20 second epochs 
        preload = True, # save in RAM to be able to edit files 
        verbose = False)
    
    #Extract epochs to data 
    data = epochs.get_data()

    subject_epoch_features =[] #Initiate empry list to keep features before averaging 
    
    for i in range(data.shape[0]): # for epoch in epochs of data 
        for ch in range (data.shape[1]): # for channels in channels of data
            segments = data[i, ch, :] # segment the data extracting one epoch and one channel from that epoch including all its timepoints 

            # Skip segments which have 0 or NAN as standard deviation 
            std = np.std(segments)
            if std == 0 or np.isnan(std):
                continue 
            else:
                segments = (segments - np.mean(segments))/np.std(segments) #Normalize (Z-score)

            # Apply features using function 
            entropy_values = shannon_entropy(segments) 
            bp_values = compute_bandpowers(segments , raw_clean.info['sfreq'])

            #Store the data as a dictionary (except bandpowers)
            featured_data = {
                "entropy" : entropy_values,
                "group": group,
                "subject": subject
            }

            # Store bandpowers in featured_data dictionary 
            featured_data.update(bp_values)
            subject_epoch_features.append(featured_data) # Append dictonary content to the second initiated list 

    #Create dataframe from list  
    df_epochs = pd.DataFrame(subject_epoch_features)

    #Average the segmented data per subject 
    df_mean = df_epochs.groupby(
        ["subject", "group"]).mean(numeric_only = True).reset_index() # indicate to only use numeric data for averaging 
    
    subject_features.append(df_mean) # Append to firstly initiated list 

# Create final data frame and concatinate to get subject labels on y axis and feature labels on x axis 
df_subject = pd.concat(subject_features, ignore_index = True)
df_subject.to_csv("features_new_EEG.csv", index = False) # Save data frame as CSV 
print(df_subject) # Investigate the data